In [1]:
import pandas as pd
import csv

path = '../crawl_data/khaosod.csv'

data = pd.read_csv(path, encoding="utf8") 
#data.shape
data.head()

,Unnamed: 0,post_page_id,post_id,post_time,post_link,post_message,post_like_count,post_share_count,post_reactor_count,post_comment_count,post_angry_count,post_love_count,post_sad_count,post_wow_count,post_haha_count,post_crawl_time,page_name
0,0,129558990394402,4333442143339378,2019-11-13 12:35:43,https://www.facebook.com/khaosod/posts/4333442...,เพื่อนบ้านรังเกียจ ลูกไม่ยอมให้เงิน เพราะกลัวจ...,1415,190,1777,205,2,10,326,19,5,2019-11-13 14:40:02.211674,khaosod
1,1,129558990394402,4333407490009510,2019-11-13 12:24:11,https://www.facebook.com/khaosod/videos/122637...,เดือดจัด!! ฟอร์จูนเนอร์ ชนแล้วหนี มอไซค์ตามไล่...,12608,3811,15338,2309,2060,6,96,364,204,2019-11-13 14:40:02.211674,khaosod
2,2,129558990394402,4333341343349458,2019-11-13 12:15:32,https://www.facebook.com/khaosod/posts/4333341...,กลุ่มผู้สูงวัย ร้องบิ๊กตู่ ขอขึ้นเบี้ยยังชีพ,1392,50,1864,504,8,4,5,40,415,2019-11-13 14:40:02.211674,khaosod
3,3,129558990394402,4333333783350214,2019-11-13 12:10:38,https://www.facebook.com/khaosod/posts/4333333...,ทุ่มทุนสร้าง เคว้ง ไทยซีรีส์เรื่องแรก บนเน็ตฟล...,304,3,310,4,0,2,0,3,1,2019-11-13 14:40:02.211674,khaosod
4,4,129558990394402,4333330193350573,2019-11-13 12:05:03,https://www.facebook.com/khaosod/posts/4333330...,ร้านเสริมสวยกระเจิง จงอางยาว 3 เมตร แผ่แม่เบี้...,349,7,385,11,0,0,0,28,8,2019-11-13 14:40:02.211674,khaosod


In [2]:
import csv

path = '../crawl_data/post interaction/khaosod_post_interact.csv'

data = pd.read_csv(path, encoding="utf8") 
#data.shape
#data.head()

#love = data.loc[data['type'] == 'Love']
#love.head()

#post_url = data.loc[data['post_url'] == 'https://www.facebook.com/khaosod/posts/4333442143339378']
#post_url.size
#post_url.head()

#user_url = data.loc[data['user_url'] == '/profile.php?id=100043671036759']
#user_url.head()

data.size

1644034

In [4]:
from tqdm import tqdm, tqdm_notebook

'''
    map[user_url][post_user_type] = user_url
'''
mapper = {}

print ("row: %d" % (data.size))

for row in tqdm_notebook(data.iterrows()):
    
    post_url = row[1]['post_url']
    type_t = row[1]['type']
    user_url = row[1]['user_url']

    if not post_url in mapper:
        mapper[post_url] = {}

    if not type_t in mapper[post_url]:
        mapper[post_url][type_t] = []
    
    mapper[post_url][type_t].append(user_url)
        
print ("Finished!")

row: 1644034



Finished!


In [5]:
len(mapper)

183

In [6]:
# Create relation between users
user_map = {}

for post_url, post_data in tqdm_notebook(mapper.items()):
    #print (post_url)
    for type_k, type_data in post_data.items():
        #print (type_k)
        #print (len(type_data))
        #print (type_data)
        for user_id_i in type_data:
            for user_id_j in type_data:
                if not user_id_i in user_map:
                    user_map[user_id_i] = {}
                if not user_id_j in user_map[user_id_i]:
                    user_map[user_id_i][user_id_j] = 0
                    
                user_map[user_id_i][user_id_j] += 1

In [9]:
w = csv.writer(open("user_map.csv", "w"))
for key, val in tqdm_notebook(user_map.items()):
    w.writerow([key, val])

In [ ]:
# Maybe, this needs csv reader to dict

In [11]:
from tqdm import tqdm, tqdm_notebook

# Get similarity set
out_data = []

for user_i, user_i_dat in tqdm_notebook(user_map.items()):
    i_action_count = len(user_map[user_i])
    
    if i_action_count < 10:
        continue
        
    for user_j in user_map[user_i]:
        
        if user_i == user_j:
            continue
            
        j_action_count = len(user_map[user_j])
        if j_action_count < 10:
            continue
        
        ij_action_count = user_map[user_i][user_j]
        if ij_action_count < 10:
            continue
        
        row = []
        row.append(user_i)
        row.append(user_j)
        row.append(ij_action_count)
        i_not_ij = i_action_count - ij_action_count
        row.append(i_not_ij)
        j_not_ij = j_action_count - ij_action_count
        row.append(j_not_ij)
        
        # Get jaccard similarity
        intersec = ij_action_count
        union = i_not_ij + ij_action_count + j_not_ij
        row.append(intersec / union)
        
        # Get overlapping coefficient
        min_ij = min((i_not_ij, j_not_ij))
        #print ('min_ij: %d' % min_ij)
        row.append(intersec / min_ij)
        
        out_data.append(row)
    
df = pd.DataFrame(out_data, columns = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"])
df.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
0,/profile.php?id=100043671036759,/profile.php?id=100042177541728,10,17886,46836,0.000154,0.000559
1,/profile.php?id=100043671036759,/suwanna.chanklin.18,15,17881,47887,0.000228,0.000839
2,/profile.php?id=100043671036759,/pongsit.prombunchu,10,17886,28634,0.000215,0.000559
3,/profile.php?id=100043671036759,/profile.php?id=100034542546272,14,17882,59353,0.000181,0.000783
4,/profile.php?id=100043671036759,/profile.php?id=100033999286026,14,17882,56013,0.000189,0.000783


In [16]:
len(out_data)

342738

In [24]:
with open('user_similarity.csv', 'w') as csvfile:
    
    # Create file writer
    w = csv.writer(csvfile, delimiter=',')
    
    # Write header
    h = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"]
    w.writerow(h)
    
    # Write row
    for i in tqdm_notebook(range(len(out_data))):
        row = out_data[i]
        w.writerow(row)

In [25]:
# Might need a file reader for user similarity

In [26]:
df = pd.DataFrame(out_data, columns = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"])
df.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
0,/profile.php?id=100043671036759,/profile.php?id=100042177541728,10,17886,46836,0.000154,0.000559
1,/profile.php?id=100043671036759,/suwanna.chanklin.18,15,17881,47887,0.000228,0.000839
2,/profile.php?id=100043671036759,/pongsit.prombunchu,10,17886,28634,0.000215,0.000559
3,/profile.php?id=100043671036759,/profile.php?id=100034542546272,14,17882,59353,0.000181,0.000783
4,/profile.php?id=100043671036759,/profile.php?id=100033999286026,14,17882,56013,0.000189,0.000783


In [28]:
#sort
df_by_overlap_coef = df.sort_values(['overlap_coef'], ascending=[0])
df_by_overlap_coef.head()

,user_i,user_j,intersec,i_not_intersec,j_not_intersec,jaccard_sim,overlap_coef
127368,/maewmeaw2560,/view.greddy,16,38348,4046,0.000377,0.003955
242528,/profile.php?id=100007268886811,/view.greddy,16,49331,4046,0.000300,0.003955
341448,/view.greddy,/profile.php?id=100007318419813,16,4046,45350,0.000324,0.003955
341446,/view.greddy,/profile.php?id=100007804512872,16,4046,55791,0.000267,0.003955
341435,/view.greddy,/profile.php?id=100041918164293,16,4046,48079,0.000307,0.003955
